In [1]:
import sys
import os
import json

sys.path.append("../../")

import datetime
from typing import Dict, List

from pipeline.extract import (
    extract_intent_upvotes,
    extract_intents,
    extract_interests,
    extract_match_stars,
    extract_recent_matches,
    extract_users,
)

from pipeline.utils.firebase import initialize_firebase_for_prefect

db = initialize_firebase_for_prefect.run(os.environ['VITE_firebase_databaseURL'],os.environ['API_ADMIN_CREDENTIALS'])



In [22]:
from pipeline.transform import (
    augment_users_with_intents,
    augment_users_with_interests,
    convert_matches_from_df,
    convert_users_from_df,
    filter_recent_matches,
    get_matching_input,
    transform_matches_for_load,
)

param_community = 'demo'
param_release = '2022-06-28'

df_users = extract_users.run(db, param_community)
df_matches = extract_recent_matches.run(db, param_community)
raw_intents = extract_intents.run(db, param_community)
raw_interests = extract_interests.run(db, param_community)
match_stars = extract_match_stars.run(db, param_community)
intent_upvotes = extract_intent_upvotes.run(db, param_community)

        # Transform extracted data to matching inputs
users_w_profile = convert_users_from_df.run(df_users)
        # TODO(vinesh, azucena): Add this transform back when it is ready
        # users_w_intents = augment_users_with_intents(
        #     users_w_profile, raw_intents
        # )
users_w_data = augment_users_with_interests.run(
    users_w_profile, raw_interests
)
matches = convert_matches_from_df.run(df_matches)
recent_matches = filter_recent_matches.run(matches, param_release)

        # Compute and validate matches
inp = get_matching_input.run(
    community=param_community,
    release=param_release,
    users=users_w_data,
    recent_matches=recent_matches,
    interests=[],
    intents=[],
    intent_upvotes=intent_upvotes,
    match_stars=match_stars,
)



[2022-06-30 14:59:06+0000] INFO - prefect | Returned 6 rows, 5 cols.
[2022-06-30 14:59:06+0000] INFO - prefect | Extracted 20 user-match records.
[2022-06-30 14:59:06+0000] INFO - prefect | Converted to 9 match records.
[2022-06-30 14:59:06+0000] INFO - prefect | Returned 9 rows, 6 cols.
[2022-06-30 14:59:06+0000] INFO - prefect | Read 9 matches.
[2022-06-30 14:59:06+0000] INFO - prefect | Filtered to 9 recent matches.


In [31]:
from pipeline.types import EngineId, Match, MatchingInput, User
from pipeline.matching.utils import get_user_group, get_user_groups_from_matches

recent_groups = get_user_groups_from_matches(inp.recent_matches)

recent_groups

{('11DBlJgn8BaV9KVZAhVAOwxFGSs1',
  'NhFa0IpZHxdElyR9PXjhnERB9t43',
  'WTosWPSnmeQZkQ6UVMuvFmZeU5p1'),
 ('11DBlJgn8BaV9KVZAhVAOwxFGSs1', 'hpKjw95CQBWUN50kmuRE0t56R0P2'),
 ('11DBlJgn8BaV9KVZAhVAOwxFGSs1', 'mt1uAbfEmzUhKSesP6tovXwLZzd2'),
 ('NhFa0IpZHxdElyR9PXjhnERB9t43',
  'PX6eEHxnSXPHhPIqvbr5xyB9Peq2',
  'ZnfAz7KaYnQNIUGYqYAFNPhzH373'),
 ('NhFa0IpZHxdElyR9PXjhnERB9t43', 'ZnfAz7KaYnQNIUGYqYAFNPhzH373'),
 ('PX6eEHxnSXPHhPIqvbr5xyB9Peq2', 'WTosWPSnmeQZkQ6UVMuvFmZeU5p1'),
 ('PX6eEHxnSXPHhPIqvbr5xyB9Peq2', 'hpKjw95CQBWUN50kmuRE0t56R0P2'),
 ('WTosWPSnmeQZkQ6UVMuvFmZeU5p1', 'hpKjw95CQBWUN50kmuRE0t56R0P2'),
 ('ZnfAz7KaYnQNIUGYqYAFNPhzH373', 'mt1uAbfEmzUhKSesP6tovXwLZzd2')}

In [29]:
for m in inp.recent_matches:
    if m.metadata != {}:
        print(m.metadata)


{'commonLetters': ['e', 'i', 'o', 's', 'y'], 'generator': 'commonLettersGenerator', 'score': 0}
{'generator': 'blank', 'score': 0}
{'commonLetters': ['a', 'l', 'o', 'r', 't'], 'generator': 'commonLettersGenerator', 'score': 0}
